<a href="https://colab.research.google.com/github/gsdos1984-sudo/simulador-moldeo/blob/main/MVP_Data_Collector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q gradio plotly pandas


In [30]:
# ================== INSTALAR DEPENDENCIAS ==================
!pip install -q gradio plotly pandas

# ================== APP: EPP Planta – Captura & Dashboard ==================
import os, pandas as pd, plotly.express as px, gradio as gr
from datetime import datetime

# ---------- PERSISTENCIA ----------
# CSV local (cámbialo a tu Drive si quieres persistencia permanente)
DATA_PATH = "/content/epp_app_data.csv"
# Ejemplo para Drive (monta antes con drive.mount):
# DATA_PATH = "/content/drive/MyDrive/epp_app_data.csv"

# Catálogos base (puedes editar)
DOWNTIME_CODES = [
    "Prep (Scheduled)", "Rest/Descanso (Scheduled)",
    "Utility/ - Low Steam Pressure/Bajo Presion de Steam", "Utility - Low Air Pressure/Baja Presion de Aire",
    "Machine/Maquina - Other/Otros", "Mold/Molde - Dirty Mold/Molde Sucio", "Mold/Molde - Ejector Rod Alignment/Alineacion del Inyector",
    "Bead Transfer/TRansferencia de Bead / Material Line Clog/ Material atascado en la Linea", "5S / Limpieza"
]
SCRAP_CODES = [
    "Adhesion", "Dimension", "Underfilling/Falta de Llenado", "Demoulding/Demolde", "Warping/Pandeo",
    "Mold - Set up (arranque)", "Process Setting/Valores de Proceso"
]

# Crear CSV si no existe
if not os.path.exists(DATA_PATH):
    pd.DataFrame(columns=[
        "ts","shift","machine","operator","product",
        "steam_bar","xsteam_fix_bar","cycle_sec",
        "e_stop_ok","ppe_ok","light_curtain_ok","door_ok","loto_ok","five_s_ok",
        "downtime_min","downtime_code","scrap_qty","scrap_code","comments"
    ]).to_csv(DATA_PATH, index=False)

# ---------- Helpers de persistencia ----------
def append_row(row: dict):
    df = pd.read_csv(DATA_PATH)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(DATA_PATH, index=False)

def empty_dash():
    """Salidas vacías del dashboard (mismo número de outputs)."""
    return gr.update(visible=False), "", None, None, None, pd.DataFrame()

# ---------- Dashboard ----------
def load_dashboard():
    if not os.path.exists(DATA_PATH):
        return empty_dash()

    df = pd.read_csv(DATA_PATH)
    if df.empty:
        return empty_dash()

    # Limpieza
    df["downtime_min"] = pd.to_numeric(df["downtime_min"], errors="coerce").fillna(0)
    df["scrap_qty"]    = pd.to_numeric(df["scrap_qty"],    errors="coerce").fillna(0)

    # KPIs
    total_rows     = len(df)
    total_downtime = int(df["downtime_min"].sum())
    total_scrap    = int(df["scrap_qty"].sum())
    avg_cycle = pd.to_numeric(df["cycle_sec"], errors="coerce").dropna()
    avg_cycle = round(avg_cycle.mean(),1) if not avg_cycle.empty else 0

    kpi = f"**Registros:** {total_rows} | **Downtime (min):** {total_downtime} | **Scrap (pzas):** {total_scrap} | **Cycle avg (s):** {avg_cycle}"

    # Gráficas
    by_code  = df.groupby("downtime_code")["downtime_min"].sum().reset_index()
    fig1 = px.bar(by_code, x="downtime_code", y="downtime_min", title="Downtime por código", text="downtime_min")
    fig1.update_layout(xaxis_title="", yaxis_title="min")

    by_scrap = df.groupby("scrap_code")["scrap_qty"].sum().reset_index()
    fig2 = px.bar(by_scrap, x="scrap_code", y="scrap_qty", title="Scrap por razón", text="scrap_qty")
    fig2.update_layout(xaxis_title="", yaxis_title="pzas")

    by_shift = df.groupby("shift")["product"].count().reset_index().rename(columns={"product":"reg"})
    fig3 = px.bar(by_shift, x="shift", y="reg", title="Registros por turno", text="reg")

    # Devuelve exactamente 6 objetos del dashboard (orden importa)
    return gr.update(visible=True), kpi, fig1, fig2, fig3, df.tail(20)

# ---------- Submit (devuelve 7 salidas: status + 6 del dashboard) ----------
def submit_form(shift, machine, operator, product,
                steam_bar, xsteam_fix_bar, cycle_sec,
                e_stop_ok, ppe_ok, light_curtain_ok, door_ok, loto_ok, five_s_ok,
                downtime_min, downtime_code, scrap_qty, scrap_code, comments):

    # Validación ejemplo: Steam 0.5–6.0 bar
    try:
        if steam_bar not in ("", None):
            steam = float(steam_bar)
            if not (0.5 <= steam <= 6.0):
                return "❌ Steam fuera de rango (0.5–6.0 bar).", *empty_dash()
    except:
        return "❌ Steam no es numérico.", *empty_dash()

    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row = {
        "ts": ts,
        "shift": shift or "",
        "machine": machine or "",
        "operator": operator or "",
        "product": product or "",
        "steam_bar": steam_bar or "",
        "xsteam_fix_bar": xsteam_fix_bar or "",
        "cycle_sec": cycle_sec or "",
        "e_stop_ok": e_stop_ok,
        "ppe_ok": ppe_ok,
        "light_curtain_ok": light_curtain_ok,
        "door_ok": door_ok,
        "loto_ok": loto_ok,
        "five_s_ok": five_s_ok,
        "downtime_min": downtime_min or 0,
        "downtime_code": downtime_code or "",
        "scrap_qty": scrap_qty or 0,
        "scrap_code": scrap_code or "",
        "comments": comments or ""
    }
    append_row(row)

    visible, kpi, fig1, fig2, fig3, tail = load_dashboard()
    return "✅ Registro enviado", visible, kpi, fig1, fig2, fig3, tail

# ================== UI GRADIO ==================
with gr.Blocks(theme=gr.themes.Soft(), title="EPP By Miguel Verastegui") as demo:
    gr.Markdown("## 📲 EPP  – By Miguel Verastegui")

    with gr.Tab("Captura-Caption"):
        with gr.Row():
            shift   = gr.Dropdown(choices=["A","B","C"], label="Turno-Shift", value="A")
            machine = gr.Textbox(label="Máquina/Machine")
            operator= gr.Textbox(label="Operador/Operator")
        product    = gr.Textbox(label="Producto-Product / Parte-Part Number")
        with gr.Row():
            steam_bar      = gr.Textbox(label="Steam (bar) 0.5–6.0")
            xsteam_fix_bar = gr.Textbox(label="X-Steam Fix (bar)")
            cycle_sec      = gr.Textbox(label="Cycle Time (sec)")

        gr.Markdown("### Checklist de seguridad (E-STOP, PPE, cortinas de luz, puertas, LOTO, 5S)")
        with gr.Row():
            e_stop_ok        = gr.Checkbox(label="E-STOP OK", value=True)
            ppe_ok           = gr.Checkbox(label="PPE OK", value=True)
            light_curtain_ok = gr.Checkbox(label="Light Curtain-Cortinas de luz OK", value=True)
            door_ok          = gr.Checkbox(label="Door-Puertas OK", value=True)
            loto_ok          = gr.Checkbox(label="LOTO OK", value=True)
            five_s_ok        = gr.Checkbox(label="5S OK", value=True)

        gr.Markdown("### Paros & Scrap (usar catálogo)")
        with gr.Row():
            downtime_min  = gr.Number(label="Downtime (min)", value=0)
            downtime_code = gr.Dropdown(choices=DOWNTIME_CODES, label="Código paro/Downtime Codes")
        with gr.Row():
            scrap_qty   = gr.Number(label="Scrap (pc)", value=0)
            scrap_code  = gr.Dropdown(choices=SCRAP_CODES, label="Razón scrap/ Scrap Reason")
        comments = gr.Textbox(label="Comentarios/Comments", lines=2)

        status = gr.Textbox(label="Estado/Status", interactive=False)
        submit = gr.Button("Enviar/Send", variant="primary")

    with gr.Tab("Dashboard"):
        dash_group = gr.Group(visible=False)
        with dash_group:
            kpi   = gr.Markdown()
            g1    = gr.Plot()
            g2    = gr.Plot()
            g3    = gr.Plot()
            table = gr.Dataframe(row_count=(5, "dynamic"), wrap=True, label="Últimos registros")

        refresh = gr.Button("🔄 Actualizar tablero-Update Dashboard")
        refresh.click(lambda: load_dashboard(), outputs=[dash_group, kpi, g1, g2, g3, table])

        # Auto-refresh cada 5 s (puedes cambiar a 10–15 s si quieres)
        auto = gr.Timer(5.0)
        auto.tick(lambda: load_dashboard(), outputs=[dash_group, kpi, g1, g2, g3, table])

    # Click del botón ENVIAR (7 salidas: status + 6 del dashboard)
    submit.click(
        submit_form,
        inputs=[shift,machine,operator,product,
                steam_bar,xsteam_fix_bar,cycle_sec,
                e_stop_ok,ppe_ok,light_curtain_ok,door_ok,loto_ok,five_s_ok,
                downtime_min,downtime_code,scrap_qty,scrap_code,comments],
        outputs=[status, dash_group, kpi, g1, g2, g3, table]
    )

    # Carga inicial del tablero al abrir
    demo.load(lambda: load_dashboard(), outputs=[dash_group, kpi, g1, g2, g3, table])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://01caed5c21f562ac22.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
